In [1]:


import sys
import json
import pickle
import pandas as pd
from transformers import (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

/Users/mahbubcseju/opt/anaconda3/envs/codebert/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('/Users/mahbubcseju/Desktop/projects/vul_detect')

In [3]:
with open("../../data/devign/test.jsonl", 'r') as f:
    data = []
    for  line in f:
        try:
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(e)


In [4]:
explanations = pickle.load(open('explanation_devign.pkl', 'rb'))

In [5]:
df_split = pd.read_csv('splits.csv')
df_map= pd.read_csv('map.csv')

In [6]:
df_split = df_split.sort_values(by='example_index')
df_map = df_map.sort_values(by='training_example_index')
df_map['example_index'] = df_map['training_example_index']

In [7]:
df = df_map.merge(df_split, how='inner', on='example_index')

In [8]:
devign_to_codexglue = {}
for item, row in df.iterrows():
    if row['split'] == 'test':
        devign_to_codexglue[row['example_index']] = row['codexglue_example_index']


In [9]:

for ex in explanations:
    ex['id'] = devign_to_codexglue[ex['id']]

In [15]:
from cmath import exp


results = []
cnt = 0
for sample in data:

    id = sample['idx']
    lines = sample['func'].split("\n")
    explan = None
    for explanation in explanations:
        if explanation['id'] == id:
            explan = explanation
            break
    # print(sample['func'])
    if explan == None:
        cnt += 1
        # print(id)
        results.append({
            'idx': id,
            'scores':[0 for i in range(len(lines))] 
        })
        continue
    assert sample['idx'] == explan['id']
    assert len(explan['edges'][0]) == len(explan['edges'][0]), "edge information are not correct"
    assert len(explan['node_code']) == len(explan['nodes']), "node information are not correct"
    # print(len(explan['nodes']), len(explan['code'].split("\n")))
    # print(explan['node_code'])
    # print(explan['code'])

    node_to_line_nos = {i: [] for i in range(len(explan['nodes']))}
    for ind_node, code in enumerate(explan['node_tokens']):
        len_ = len(code)
        line_tokens = [[] for i in range(len(lines))]
        for ind_token, token in enumerate(code):
            for ind_line, line in enumerate(lines):
                # print(token, line)
                if token in line:
                    line_tokens[ind_line].append(ind_token)

        for ind_line in range(len(lines)):
            if len(list(set(line_tokens[ind_line]))) == len_:
                # print(ind_node, ind_line)
                # print(explan['node_code'][ind_node], lines[ind_line])
                node_to_line_nos[ind_node].append(ind_line)
        # break
    # print(node_to_line_nos)
    line_scores = [[] for line in lines]
    for u, v, score in zip(explan['edges'][0], explan['edges'][1], explan['edge_importance']):
        for node in node_to_line_nos[u] + node_to_line_nos[v]:
            line_scores[node].append(score)
            # print(node, score)
    
    results.append({
        'idx': sample['idx'],
        'scores': [sum(line_score) for line_score in line_scores]
    })
    # print([sum(line_score)/len(line_score) if line_score else 0 for line_score in line_scores])
    # for line_score in line_scores:
    #     print(line_score)


In [16]:
cnt

293

In [17]:
json.dump(results, open("line_scores.json", 'w'))